https://blog.csdn.net/qq_39309652/article/details/90399674

In [46]:
import pandas as pd
import numpy as np
import re

In [47]:
from bs4 import BeautifulSoup

In [48]:
def review_to_wordlist(review):
    '''
    把IMDB的评论转成词序列
    参考：http://blog.csdn.net/longxinchen_ml/article/details/50629613
    '''
    # 去掉HTML标签，拿到内容
    review_text = BeautifulSoup(review, "html.parser").get_text()
    # 用正则表达式取出符合规范的部分
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 小写化所有的词，并转成词list
    words = review_text.lower().split()
    # 
    new_review = " ".join(words)
    return new_review

In [49]:
a=review_to_wordlist("Tu aa to sae dekh kia kia sa??? pilati😂😎😋sss:)ss.…  ")
print(a)

tu aa to sae dekh kia kia sa pilati sss ss


In [50]:
# 载入数据集
train = pd.read_csv('../data/train.csv', header=0)
test = pd.read_csv('../data/20190527_test.csv', header=0)
print (train.head())
print (test.head())

   ID                                             review     label
0   1                       Jo bhi ap se tou behtar hoon  Negative
1   2          ya Allah meri sister Affia ki madad farma  Positive
2   3  Yeh khud chahta a is umar main shadi krna.  ha...  Negative
3   4        Tc ? Apky mun xe exe alfax achy nae lgty 😒💃  Negative
4   5                                               Good  Positive
   ID                                          review
0   1                              Phr tissuw se saaf
1   2      Jail Road Per Firing Se 1 Shakhs Janbahaq 
2   3                         mehfil loot li aunty ne
3   4  Rehnay do butt sahb nay galiya boht deni hain 
4   5                                      Zabardast 


In [51]:
train_data = train.review.apply(review_to_wordlist).tolist()
test_data = test.review.apply(review_to_wordlist).tolist()

In [65]:
# 预览数据
print (train_data[:10])
print (test_data[:10])
print(len(train_data))
print(len(test_data))

['jo bhi ap se tou behtar hoon', 'ya allah meri sister affia ki madad farma', 'yeh khud chahta a is umar main shadi krna had ogi', 'tc apky mun xe exe alfax achy nae lgty', 'good', 'american president john f kennedy aur in ke bhai robert kennedy se bhi marilyn monroe ke affairs ka charcha raha', 'commission aur kickback ka dor dora raha quomi assase koriyon ke mol farokhat keye gaye', 'allah pak nazer e bd sy bechye or humesha bohat tarkian dy ameeen', 'amoman log samajhte hain ke jhok siyal hi abid ali ki pheli serial thi halan ke sach ye hai ke unhon ne sab se phele saleem chishtti le likhe howe kheel zanjeer mein adakari ki', 'akki khanyani k tum v good wesy tum shkll sy bhi khesiani billi kgti ho']
['phr tissuw se saaf', 'jail road per firing se shakhs janbahaq', 'mehfil loot li aunty ne', 'rehnay do butt sahb nay galiya boht deni hain', 'zabardast', 'ap jahil ho', 'phir tm sab kahti ho k larko ko hamri id kahan se milti ha yu khula tag marogi to rozay me hamri eid hi hogi p p', 'b

In [53]:
def encode_label(text):
    if(text=="Positive"):
        return 1
    else:
        return 0

In [54]:
label = train.label.apply(encode_label).tolist()

In [84]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
# 参考：http://blog.csdn.net/longxinchen_ml/article/details/50629613
tfidf = TFIDF(min_df=2, # 最小支持度为2
           max_features=None,
           strip_accents='unicode',
           analyzer='word',
           token_pattern=r'\w{1,}',
           ngram_range=(1, 3),  # 二元文法模型
           use_idf=1,
           smooth_idf=1,
           sublinear_tf=1,
           stop_words = 'english') # 去掉英文停用词

# 合并训练和测试集以便进行TFIDF向量化操作
data_all = train_data + test_data
len_train = len(train_data)

tfidf.fit(data_all)
data_all = tfidf.transform(data_all)
# 恢复成训练集和测试集部分
train_x = data_all[:len_train]
test_x = data_all[len_train:]
print ('TF-IDF处理结束.')

TF-IDF处理结束.


In [85]:
from sklearn.naive_bayes import MultinomialNB as MNB

model_NB = MNB()
model_NB.fit(train_x, label)
MNB(alpha=1.0, class_prior=None, fit_prior=True)

from sklearn.model_selection import cross_val_score
import numpy as np

print ("多项式贝叶斯分类器10折交叉验证得分: ", np.mean(cross_val_score(model_NB, train_x, label, cv=10, scoring='roc_auc')))

多项式贝叶斯分类器10折交叉验证得分:  0.8631329048865058


In [81]:
test_predicted = np.array(model_NB.predict_proba(test_x))

In [82]:
print(test_predicted[:10])

[[0.63226051 0.36773949]
 [0.97693053 0.02306947]
 [0.59545547 0.40454453]
 [0.62338105 0.37661895]
 [0.12465368 0.87534632]
 [0.85289353 0.14710647]
 [0.53291268 0.46708732]
 [0.57255214 0.42744786]
 [0.46348977 0.53651023]
 [0.67855174 0.32144826]]


In [83]:
df = pd.DataFrame(zip(test.ID,test_predicted[:,1]), columns=["ID","Pred"])
df.to_csv('result.csv', index=False, float_format='%.6f')